In [ ]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import time
import datetime


In [ ]:
#for startup
BasicLink1="http://www.canada411.ca/search/si/"
PageNumber=0
BasicLink2="/"
BasicLink3="/Canada/"

In [ ]:
#Reading Names from client's file
with open('names.txt', 'r') as myfile:
    data=myfile.read()
all_names=data.split(",")

len(all_names)


In [ ]:
#Scrapping Total number of names for a specific name from first page
def GetNumberofNames(name):
    no_name=True
    link=""
    NumberOfNames=0
    try:
        link=BasicLink1 + str(1) + BasicLink2 + name + BasicLink3
        r=requests.get(link)
        soup=BeautifulSoup(r.text,'html.parser')
        resultstop= soup.find_all('div',attrs={'class':'c411ResultsTop'})
        fr = resultstop[0]
        x = fr.find("h1").text
        y = x.split(" ")
        NumberOfNames = y[0]
        NumberOfNames=NumberOfNames.replace(",","")
        NumberOfNames=int(NumberOfNames)
        no_name=False
    except:
        NumberOfNames=0
        no_name=True
    return NumberOfNames,no_name

In [ ]:
#Writing in 2 seperate files, "ALL Names.txt" has all names and "ALL Contact Numbers.txt" has all contact numbers 
def WriteInFile(Names,ContactNumbers,NumberOfNames):
    file=open("ALL Names.txt","a")
    file2=open("ALL Contact Numbers.txt","a")
    for i in range(NumberOfNames):
        file.write(Names[i])
        file.write("\n")
        file2.write(ContactNumbers[i])
        file2.write("\n")


In [ ]:
#start_time and end_time , total_time : to calculate execution time
#All_names_Done: Number of names done in total
#Names_fetched_for_this_name: number of names fetched for one name
#all_names: all names to be searched 
start_time = datetime.datetime.now().time().strftime('%H:%M:%S')

All_names_Done=0
Names_fetched=0
Names_fetched_for_this_name=0
for name in all_names: 
    start_time2 = datetime.datetime.now().time().strftime('%H:%M:%S')
    number_fetch_bool=False
    NumberOfNames,no_name=GetNumberofNames(name)
    PageNumber=0
    Scrapped_Names = []
    Scrapped_ContactNumbers=[]
    Done=0
    j=0
    Names_fetched_for_this_name=0
    if no_name==False:
        for i in range(NumberOfNames):
            if Done%15==0:
                PageNumber+=1
                link=BasicLink1 + str(PageNumber) + BasicLink2 + name+ BasicLink3
                r=requests.get(link)
                soup=BeautifulSoup(r.text,'html.parser')
                ContactList= soup.find_all('span',attrs={'class':'c411Phone'})
                NamesList  = soup.find_all('h2',attrs={'class':'c411ListedName'})
                j=0
            Scrapped_Names.append(NamesList[j].text)
            Scrapped_ContactNumbers.append(ContactList[j].text)
            Done+=1
            j+=1
            Names_fetched+=1
            Names_fetched_for_this_name+=1
        All_names_Done+=1
        WriteInFile(Scrapped_Names,Scrapped_ContactNumbers,NumberOfNames)

        if NumberOfNames==Names_fetched_for_this_name:
            number_fetch_bool=True            
        print(All_names_Done,"\t", NumberOfNames ,"\t", Names_fetched_for_this_name , "\t" , number_fetch_bool , "\t" ,Names_fetched , "\t" , name, "\t")
end_time = datetime.datetime.now().time().strftime('%H:%M:%S')
total_time=(datetime.datetime.strptime(end_time,'%H:%M:%S') - datetime.datetime.strptime(start_time,'%H:%M:%S'))
print(total_time)